# Simple kriging

In [76]:
# import libraries

import numpy as np
import pandas as pd
import lasio
import math

from skgstat import models, Variogram
from skgstat.models import spherical

from scipy.optimize import curve_fit
from scipy.spatial.distance import squareform, pdist

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [77]:
# read data

WELL_NAME = 'new_excel'
df = pd.read_excel('data/'+WELL_NAME+'.xlsx')
df = df.drop(columns=['FACIES', 'PERM', 'AI'])
df

,X,Y,PORO
0,100,900,0.100187
1,100,800,0.107947
2,100,700,0.085357
3,100,600,0.108460
4,100,500,0.102468
...,...,...,...
256,780,669,0.175642
257,50,779,0.091725
258,430,589,0.139057
259,130,489,0.099351


In [78]:
df['RESIDUAL'] = df.PORO - df.PORO.mean()
df

,X,Y,PORO,RESIDUAL
0,100,900,0.100187,-0.050170
1,100,800,0.107947,-0.042410
2,100,700,0.085357,-0.065000
3,100,600,0.108460,-0.041897
4,100,500,0.102468,-0.047889
...,...,...,...,...
256,780,669,0.175642,0.025285
257,50,779,0.091725,-0.058632
258,430,589,0.139057,-0.011300
259,130,489,0.099351,-0.051006


In [105]:
def simple_kriging(data, unknown_index):
    data_corrupted = data[data.index != unknown_index]
    
    #V = Variogram(data_corrupted[['X', 'Y']], data_corrupted['PORO'], model='gaussian', normalize=False, n_lags=15)
    #V.estimator = 'matheron'
    #V.model = 'spherical'

    #xdata = V.bins
    #ydata = V.experimental
    #print(xdata, ydata)

    #cof, cov = curve_fit(models.spherical, xdata, ydata)
    
    cof = [200, 1, 0]
    
    ar1 = np.sqrt((data_corrupted['X'] - data['X'].loc[unknown_index])**2 \
                  + (data_corrupted['Y'] - data['Y'].loc[unknown_index])**2)
    ar2 = np.array(data_corrupted[['X', 'Y']])
    
    distance_matrix = squareform(pdist(ar2))
    distance_mat = np.zeros((distance_matrix.shape[0], distance_matrix.shape[1]+1))
    distance_mat[:, :-1] = distance_matrix
    distance_mat[:, -1] = ar1
    
    model = lambda h: spherical(h, cof[0], cof[1], cof[2])
    
    var_matrix = np.zeros_like(distance_mat)
    for i in range(var_matrix.shape[0]):
        for j in range(var_matrix.shape[1]):
            if distance_mat[i, j] > cof[0]:
                val = 1
            else:
                val = 1.5 * (distance_mat[i, j] / cof[0]) - 0.5 * (distance_mat[i, j] / cof[0])**3
            var_matrix[i, j] = cof[2] + cof[1] * val
            
    cov_matrix = np.zeros_like(distance_mat)
    for i in range(cov_matrix.shape[0]):
        for j in range(cov_matrix.shape[1]):
            cov_matrix[i, j] = cof[1] + cof[2] - var_matrix[i, j]
            
    inv_left_side = np.linalg.inv(cov_matrix[:, :-1])
    
    W = inv_left_side @ cov_matrix[:, -1]
    W_sum = np.sum(W)
    W_mean = np.mean(W)
    
    kriging_est = np.sum(data_corrupted['RESIDUAL'] * W) + data_corrupted['PORO'].mean()
    kriging_var = cof[1] + cof[2] - np.sum(W * cov_matrix[:,-1])
    
    results.append(kriging_est)
    variations.append(kriging_var)
    
    return results, variations

In [106]:
results = []
variations = []

for i in df.index:
    res, var = simple_kriging(df, unknown_index=i)

In [107]:
df['KRIGING'] = results
df

,X,Y,PORO,RESIDUAL,KRIGING
0,100,900,0.100187,-0.050170,0.142411
1,100,800,0.107947,-0.042410,0.110964
2,100,700,0.085357,-0.065000,0.117875
3,100,600,0.108460,-0.041897,0.101163
4,100,500,0.102468,-0.047889,0.108564
...,...,...,...,...,...
256,780,669,0.175642,0.025285,0.168718
257,50,779,0.091725,-0.058632,0.118573
258,430,589,0.139057,-0.011300,0.128572
259,130,489,0.099351,-0.051006,0.098696


In [108]:
mae = np.mean(np.abs((df['KRIGING'] - df['PORO'])))
print('MAE {} for field porosity'.format(mae))

MAE 0.0113880071377563 for field porosity


In [109]:
mse = np.mean((df['KRIGING'] - df['PORO'])**2)
print('MSE {} for filed porosity'.format(mse))

MSE 0.00025234756391852786 for filed porosity


In [110]:
np.var(df['PORO'].values) - np.var(df['KRIGING'].values)

0.00038839881647537806

In [111]:
print('Estemated porosity mean {}, standart dev {} , varience {} for field'.format(df['KRIGING'].mean(), df['KRIGING'].std(), df['KRIGING'].var()))

Estemated porosity mean 0.15312882478848122, standart dev 0.04569935649321963 , varience 0.002088431183894375 for field


In [112]:
print('Initial porosity mean {}, standart dev {} , varience {} for field'.format(df['PORO'].mean(), df['PORO'].std(), df['PORO'].var()))

Initial porosity mean 0.15035706160196521, standart dev 0.049782766515849446 , varience 0.0024783238419715807 for field
